# Błędy numeryczne

Zagadnienia:
* Reprezentacja liczb w komputerze:
    * Zapis stałoprzecinkowy,
    * Zapis zmiennoprzecinkowy (standard IEEE 754),
* Błędy numeryczne:
    * Błąd względny i bezwzględny.
    * Błąd zaokrąglenia (ang. *round-off error*).
    * Błąd obcięcia (ang. *truncation error*).
    * Błędy związane z przyjętym sposobem rozwiązywania.

## Błąd względny i bezwzględny

***Zadanie 1.***  
Liczbę $\textrm{e}$ możemy zdefiniować jako $\sum\limits_{n=0}^{\infty} \frac{1}{n!}$. Oblicz błąd względny i bezwzględny aproksymacji liczby $\textrm{e}$ poprzez $\textrm{e}^{*}$ w przypadku gdy:  
* $\textrm{e}_1^{*}=\sum\limits_{n=0}^{5} \frac{1}{n!}$    

* $\textrm{e}_2^{*}=\sum\limits_{n=0}^{10} \frac{1}{n!}$  

* $\textrm{e}_3^{*}=\sum\limits_{n=0}^{20} \frac{1}{n!}$  

In [16]:
import numpy as np
import math

def aproksymacja(liczba_n):
    return sum(1 / math.factorial(n) for n in range(liczba_n + 1))

def blad_bez(n):
    return abs(np.e - aproksymacja(n))

def blad_wzg(n):
    return blad_bez(n)/abs(np.e)

print(f'Liczba e: {np.e}')
print(f'e1:{aproksymacja(5)}; Błąd bezwzględny: {blad_bez(5)}; Błąd względny: {blad_wzg(5)}')
print(f'e2:{aproksymacja(10):.15f}; Błąd bezwzględny: {blad_bez(10):.15f}; Błąd względny: {blad_wzg(10):.15f}')
print(f'e3:{aproksymacja(20)}; Błąd bezwzględny: {blad_bez(20):.15f}; Błąd względny: {blad_wzg(20):.15f}')

Liczba e: 2.718281828459045
e1:2.716666666666667; Błąd bezwzględny: 0.0016151617923783057; Błąd względny: 0.0005941848175815963
e2:2.718281801146385; Błąd bezwzględny: 0.000000027312661; Błąd względny: 0.000000010047766
e3:2.718281828459045; Błąd bezwzględny: 0.000000000000000; Błąd względny: 0.000000000000000


## Zapis zmiennoprzecinkowy

**Uwaga**: Używane tu pojęcie **cecha** ma inne znaczenie, niż poznane być może w szkole – skrótowo: „część całkowita”. Dlatego poniżej dodawane jest inne określenie – „wykładnik”.

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import sys

In [18]:
print(np.finfo(float))
eps = np.finfo(float).eps

Machine parameters for float64
---------------------------------------------------------------
precision =  15   resolution = 1.0000000000000001e-15
machep =    -52   eps =        2.2204460492503131e-16
negep =     -53   epsneg =     1.1102230246251565e-16
minexp =  -1022   tiny =       2.2250738585072014e-308
maxexp =   1024   max =        1.7976931348623157e+308
nexp =       11   min =        -max
smallest_normal = 2.2250738585072014e-308   smallest_subnormal = 4.9406564584124654e-324
---------------------------------------------------------------



***Zadanie 2.***


a) Wstaw dowolną wartość do liczby x, np. 0.1. Które liczby mają tę samą mantysę, a które tę samą cechę (wykładnik)? Sprawdź wypisując ich wartości w formacie hex.


**Uwaga:** Do odczytania liczby w formacie szesnastkowym możesz wykorzystać funkcję:

In [23]:
1.25.hex()

'0x1.4000000000000p+0'

In [20]:
5000.0.hex()

'0x1.3880000000000p+12'

lub

In [21]:
import struct
def double_to_hex(f):
    print(hex(struct.unpack('<Q', struct.pack('<d', f))[0]))

In [22]:
double_to_hex(1.25)

0x3ff4000000000000


Zamianę w drugą stronę można przeprowadzić za pomocą funkcji *int*

In [7]:
int('3ff', 16)

1023

In [33]:
import struct

def double_to_binary(f):
    return bin(struct.unpack('<Q', struct.pack('<d', f))[0])[2:].zfill(64)

def find_same_mantissa_exponent(x):
    x_hex = x.hex()
    x_binary = double_to_binary(x)
    exponent = x_binary[1:12]
    mantissa = x_binary[12:]
    
    print(f"Liczba {x} w formacie hex: {x_hex}")
    print(f"Reprezentacja binarna liczby {x}: {x_binary}")
    print(f"Exponent: {exponent}")
    print(f"Mantysa: {mantissa}")

    same_mantissa = []
    same_exponent = []
    
    for i in range(2**10):
        test_number = float.fromhex(x_hex[0] + hex(i)[2:].zfill(3) + x_hex[4:])
        test_binary = double_to_binary(test_number)
        test_exponent = test_binary[1:12]
        test_mantissa = test_binary[12:]

        if test_mantissa == mantissa:
            same_mantissa.append(test_number)
        if test_exponent == exponent:
            same_exponent.append(test_number)
    
    print(f"Liczby o takiej samej mantysie jak {x}: {same_mantissa}")
    print(f"Liczby o takim samym wykładniku jak {x}: {same_exponent}")

# Przykładowe użycie
x = 0.1
find_same_mantissa_exponent(x)

Liczba 0.1 w formacie hex: 0x1.999999999999ap-4
Reprezentacja binarna liczby 0.1: 0011111110111001100110011001100110011001100110011001100110011010
Exponent: 01111111011
Mantysa: 1001100110011001100110011001100110011001100110011010
Liczby o takiej samej mantysie jak 0.1: [450359962737049.6, 7205759403792794.0, 1.152921504606847e+17]
Liczby o takim samym wykładniku jak 0.1: []




b) Porównaj zapis liczb 0.1, 0.2 i 0.3 w formacie zmiennoprzecinkowym.

* Czy można przewidzieć, czy suma 0.1 + 0.2 będzie reprezentowana dokładnie tak samo jak liczba 0.3?
* Jaki będzie wynik porównania tej sumy z liczbą 0.3 i konsekwencje użycia warunku równościowego / nierównościowego w pętli?



In [ ]:
a = 0.1
b = 0.2
c = 0.3

print(f'a = {float(a).hex()}')
print(f'b = {float(b).hex()}')
print(f'c = {float(c).hex()}')

print(f'c = {c}')
print("a + b =", a + b)
print(f'Czy a + b równa się c?: {a + b == c}')

a = 0x1.999999999999ap-4
b = 0x1.999999999999ap-3
c = 0x1.3333333333333p-2
c = 0.3
a + b = 0.30000000000000004
Czy a + b równa się c?: False
(a + b) - c = 5.551115123125783e-17


c) Przyjmijmy, że „skrótowe” określenie *liczba dokładnie reprezentowana* oznacza liczbę, która jest reprezentowana w formacie zmiennoprzecinkowym dokładnie, tzn. bez konieczności zaokrąglania.
* Ile jest liczb dokładnie reprezentowanych w przedziale $[1, 2)$?
* Zaobserwuj (z pomocą formatu hex) jakie wartości mają bity w polu mantysy w przypadku liczby 1.0.
* Jaką liczbę trzeba dodać do 1.0, aby tylko na najmłodszej pozycji pola mantysy pojawiło się 1?
* Czy wyżej otrzymana suma jest najmniejszą liczbą dokładnie reprezentowana, spośród liczb większych od 1?
* Jak przypuszczasz, jakie wartości będą miały poszczególne bity mantysy w przypadku największej liczby dokładnie reprezentowanej, której cecha (wykładnik) jest równy 0? Sprawdź, czy tak jest rzeczywiście – wpisując wartość tej liczby (jaka to jest wartość?).


In [46]:
liczba_dokladnie_reprezentowana = 2**52
print("Liczba dokładnie reprezentowana w przedziale [1, 2):", liczba_dokladnie_reprezentowana)

wartosc_mantysy_1 = float.hex(1.0).split('p')[0].split('.')[-1]
print("Wartość bitów pola mantysy dla liczby 1.0:", wartosc_mantysy_1)

dodana_wartosc = 2**-52
suma = 1.0 + dodana_wartosc

print("Suma 1.0 + 2^-52:", suma)

najwieksza_mantysa_wykładnik_0 = '1.' + '1' * 52
najwieksza_liczba = float.fromhex(najwieksza_mantysa_wykładnik_0 + 'p0')

print("Największa liczba dokładnie reprezentowana, wykładnik=0:", najwieksza_liczba)

Liczba dokładnie reprezentowana w przedziale [1, 2): 4503599627370496
Wartość bitów pola mantysy dla liczby 1.0: 0000000000000
Suma 1.0 + 2^-52: 1.0000000000000002
Największa liczba dokładnie reprezentowana, wykładnik=0: 1.0666666666666667


d) Oblicz długości przedziałów, do których należą wszystkie liczby mające cechę (wykładnik) równy: 0, 1, 2, 5, -1, -3.

* Czy w każdym z tych przedziałów jest tyle samo liczb dokładnie reprezentowanych? Jeżeli nie, to ile w każdym z nich?
* Jaka jest odległość między dwoma sąsiednimi liczbami dokładnie reprezentowanymi (odległość, czyli różnica ich wartości) w każdym z tych przedziałów?
* Jaki jest maksymalny błąd bezwzględny zaokrąglenia w każdym z tych przedziałów?
* Jaki jest maksymalny błąd względny zaokrąglenia w każdym z tych przedziałów?





In [ ]:
import math

def liczba_dokladnie_reprezentowana(dolny_przedzial, gorny_przedzial):
    return int((2**52) * (gorny_przedzial - dolny_przedzial))

def odleglosc_miedzy_liczbami():
    return 2**-52

def maksymalny_blad_bezwzgledny():
    return 2**-53

def maksymalny_blad_wzgledny(dolny_przedzial, gorny_przedzial):
    return (2**-52) / dolny_przedzial

cechy = [0, 1, 2, 5, -1, -3]
for cecha in cechy:
    dolny_przedzial = 2**cecha
    gorny_przedzial = 2**(cecha + 1)
    liczba_dokladnie_reprezentowana_w_przedziale = liczba_dokladnie_reprezentowana(dolny_przedzial, gorny_przedzial)
    odleglosc_miedzy_liczbami_dokladnie_reprezentowanymi = odleglosc_miedzy_liczbami()
    maksymalny_blad_bezwzgledny_zaokraglenia = maksymalny_blad_bezwzgledny()
    maksymalny_blad_wzgledny_zaokraglenia = maksymalny_blad_wzgledny(dolny_przedzial, gorny_przedzial)

    print(f"Cecha {cecha}:")
    print(f"Długość przedziału: ({dolny_przedzial}, {gorny_przedzial})")
    print(f"Liczba dokładnie reprezentowana w przedziale: {liczba_dokladnie_reprezentowana_w_przedziale}")
    print(f"Odległość między dwoma sąsiednimi liczbami dokładnie reprezentowanymi: {odleglosc_miedzy_liczbami_dokladnie_reprezentowanymi}")
    print(f"Maksymalny błąd bezwzględny zaokrąglenia: {maksymalny_blad_bezwzgledny_zaokraglenia}")
    print(f"Maksymalny błąd względny zaokrąglenia: {maksymalny_blad_wzgledny_zaokraglenia}")
    print()


Cecha 0:
Długość przedziału: (1, 2)
Liczba dokładnie reprezentowana w przedziale: 4503599627370496
Odległość między dwoma sąsiednimi liczbami dokładnie reprezentowanymi: 2.220446049250313e-16
Maksymalny błąd bezwzględny zaokrąglenia: 1.1102230246251565e-16
Maksymalny błąd względny zaokrąglenia: 2.220446049250313e-16

Cecha 1:
Długość przedziału: (2, 4)
Liczba dokładnie reprezentowana w przedziale: 9007199254740992
Odległość między dwoma sąsiednimi liczbami dokładnie reprezentowanymi: 2.220446049250313e-16
Maksymalny błąd bezwzględny zaokrąglenia: 1.1102230246251565e-16
Maksymalny błąd względny zaokrąglenia: 1.1102230246251565e-16

Cecha 2:
Długość przedziału: (4, 8)
Liczba dokładnie reprezentowana w przedziale: 18014398509481984
Odległość między dwoma sąsiednimi liczbami dokładnie reprezentowanymi: 2.220446049250313e-16
Maksymalny błąd bezwzględny zaokrąglenia: 1.1102230246251565e-16
Maksymalny błąd względny zaokrąglenia: 5.551115123125783e-17

Cecha 5:
Długość przedziału: (32, 64)
Lic

***Zadanie 3.***

Spójrz na poniższe zjawiska, zastanów się nad ich przyczynami.

***Problem skali***

Jaki wynika da poniższy kod:

In [48]:
a = 1.0
b = 0.0000000000000000000000001
c = a + b
print(f'{c:.17f}')

1.00000000000000000


***Problem reprezentacji w zapisie binarnym*** 

In [49]:
if 0.1+0.2==0.3:
    print("True")
else:
    print("False")

False


In [50]:
suma = 0
for i in range(0,100):
    suma += 0.1
print(f'{suma:.15f}')

9.999999999999980


**Zadanie 4.**

Niech:
* $a=0.3$
* $b=10^{-13}$
* $c=a-b$
* $d=a-c$

Czy $b$ i $d$ są sobie równe w sensie analitycznym? Co z przypadkiem numerycznym? Zdefiniuj odpowiednie zmienne i porównaj wyniki. Porównaj szesnastkowe i dziesietne reprezentacje zmiennych `b` i `d`. Ile cyfr w reprezentacji `d` możemy uznać za wiarygodne?

In [51]:
a = 0.3
b = 10**-13
c = a - b
d = a + c

print("Wartość zmiennej b:", b)
print("Wartość zmiennej d:", d)

print("\nReprezentacja szesnastkowa zmiennej b:", float(b).hex())
print("Reprezentacja dziesiętna zmiennej b:", format(b, '.20f'))
print("Reprezentacja szesnastkowa zmiennej d:", float(d).hex())
print("Reprezentacja dziesiętna zmiennej d:", format(d, '.20f'))

Wartość zmiennej b: 1e-13
Wartość zmiennej d: 0.5999999999999

Reprezentacja szesnastkowa zmiennej b: 0x1.c25c268497682p-44
Reprezentacja dziesiętna zmiennej b: 0.00000000000010000000
Reprezentacja szesnastkowa zmiennej d: 0x1.3333333332faep-1
Reprezentacja dziesiętna zmiennej d: 0.59999999999989994670


**Zadanie 5.**

Rozważmy macierz kwadratową:
$$ A=\left[\begin{array}{ccc}
\sqrt{2} & \frac{1}{7}\\
\pi\sqrt{2} &\frac{\pi}{7}\\
\end{array}\right]
$$
* Czy jest to macierz osobliwa? Jak to sprawdzić?
* Oblicz wyznacznik tej macierzy w sposób analityczny i numeryczny (`np.linalg.det`). Czy otrzymałeś te same wartości? Czy wyniki obliczeń numerycznych prowadzą do poprawnej odpowiedzi na pierwsze pytanie? Jakie może to mieć skutki?

In [52]:
import numpy as np

A = np.array([[np.sqrt(2), 1/7], [np.pi * np.sqrt(2), np.pi/7]])

det_analitycznie = (np.sqrt(2) * np.pi/7) - (1/7 * np.pi * np.sqrt(2))

det_numerycznie = np.linalg.det(A)

if det_numerycznie == 0:
    print("Macierz A jest osobliwa.")
else:
    print("Macierz A nie jest osobliwa.")

print("Wyznacznik macierzy obliczony analitycznie:", det_analitycznie)
print("Wyznacznik macierzy obliczony numerycznie:", det_numerycznie)


Macierz A nie jest osobliwa.
Wyznacznik macierzy obliczony analitycznie: 0.0
Wyznacznik macierzy obliczony numerycznie: 4.972970853216942e-17


**Zadanie 6.**

Jedną z metod rozwiązywania równań nieliniowych jest metoda Newtona. Metoda ta powtarza obliczenia, aż spełniony nie będzie warunek stopu. Jeżeli spełnione są odpowiednie założenia to metoda ta zbiega do rozwiązania. Załóżmy, że warunki są spełnione i rozważmy funkcję $f(x)=cos(x)-x$. Poniżej znajdziesz implementację metody Newtona, w której brakuje warunku stopu. Spróbuj go uzupełnić. Warunek powinien być spełniony w momencie, w którym chcemy zakończyć działanie procedury.

Z matematycznego punktu widzenia znajdujemy się w zerze, jeżeli kolejne przybliżenia uzyskane z metody Newtona nie różnią się od siebie (np. dwa ostatnie). Zaimplementuj taki warunek i sprawdź, czy program zatrzyma się. Czy taki warunek stopu jest bezpieczny i można go stosować uniwersalnie? Jak można go poprawić?

In [53]:
def newton(f, f_prim, x_0):
    x_prev = x_0
    while True:
        x_n=x_prev-(f(x_prev)/f_prim(x_prev))
        print(f'x_0={x_n}')
        double_to_hex(x_n)
        
        if True: # Zastąp True swoim warunkiem stopu
            return x_n
        else:
            x_prev=x_n

In [54]:
def f(x):
    return # Uzupełnij definicję funkcji 

def f_prim(x):
    return # Uzupełnij definicję pochodnej

In [ ]:
def newton(f, f_prim, x_0, tol=1e-8, max_iter=100):
    x_prev = x_0
    for i in range(max_iter):
        x_n = x_prev - (f(x_prev) / f_prim(x_prev))
        print(f'Iteracja {i + 1}: x = {x_n}')
        
        if abs(x_n - x_prev) < tol:
            return x_n
        else:
            x_prev = x_n

    return x_n

def f(x):
    return math.cos(x) - x

def f_prim(x):
    return -math.sin(x) - 1 

**Zadanie domowe**  

Dany jest ciąg:  
  
$G(p)=\cfrac{1}{p}\left[10^{p}\left(1+p\pi^{*} \cdot 10^{-p}\right)-10^{p}\right],\ p=1,2,3,\dots,n $,
w którym $\pi^{*}$ jest przybliżeniem liczby $\pi$ z dokładnością do 15 miejsc po przecinku.
  
Stwórz funkcję, która umożliwi obliczenie $n$ pierwszych wyrazów tego ciągu i zwróci je w postaci wektora (do przybliżenia wartości $\pi$ możesz wykorzystać funkcję *[round](https://numpy.org/doc/stable/reference/generated/numpy.round_.html)*). Oblicz błąd bezwzględny i błąd względny otrzymanych wyników. Przedstaw rezultaty na wykresie w zależności od $n$. Wartości błędów przedstaw w skali logarytmicznej (funkcja *plt.semilogy()*).

Przeanalizuj rozwiązanie i odpowiedz na następujące pytania:  
* Czy w obliczeniach analitycznych wartość $G(p)$ zależy od $p$?
* Jak zmienia się błąd w zależności od $n$? O czym to świadczy?
* Z jakim/jakimi rodzajami błędu mamy do czynienie?
